In [2]:
!pip install kagglehub
!pip install iterative-stratification


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("xhlulu/vinbigdata-chest-xray-resized-png-256x256")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [5]:
!mv /root/.cache/kagglehub/datasets/xhlulu/vinbigdata-chest-xray-resized-png-256x256/versions/1 /workspace/256data

In [3]:
!pip install pandas numpy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple, Optional

NO_FINDING = 14

def warp_box(box, sx, sy): #scale factor x, scale factor y
    if box is None: return None
    x1,y1,x2,y2 = box
    return [x1*sx, y1*sy, x2*sx, y2*sy]

def clamp_box(box, w, h):
    if box is None: return None
    x1,y1,x2,y2 = box
    x1 = max(0.0, min(w, x1)); y1 = max(0.0, min(h, y1))
    x2 = max(0.0, min(w, x2)); y2 = max(0.0, min(h, y2))
    if x2 <= x1 or y2 <= y1:
        return None
    return [x1,y1,x2,y2]

def transform_boxes_to_224(
    df: pd.DataFrame,
    mode: str = "from_256",  # or "from_original"
    per_image_src_size: Optional[Dict[str, Tuple[int,int]]] = None,
    class_no_finding: int = NO_FINDING,
    drop_degenerate: bool = True,
):
    """
    Expects columns: image_id, class_id, x_min, y_min, x_max, y_max
    Returns a copy with boxes mapped to a 224x224 image via warp.
    """
    req_cols = {"image_id","class_id","x_min","y_min","x_max","y_max"}
    missing = req_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    out = df.copy()
    w_target = h_target = 224
    scale_256_to_224 = w_target / 256.0  # == 224/256

    new_x1,new_y1,new_x2,new_y2,keep = [],[],[],[],[]

    for row in out.itertuples(index=False):
        img = getattr(row, "image_id")
        cls = getattr(row, "class_id")
        x1 = getattr(row, "x_min"); y1 = getattr(row, "y_min")
        x2 = getattr(row, "x_max"); y2 = getattr(row, "y_max")

        # Keep NaNs for No finding
        if cls == class_no_finding or any(pd.isna(v) for v in (x1,y1,x2,y2)):
            new_x1.append(np.nan); new_y1.append(np.nan)
            new_x2.append(np.nan); new_y2.append(np.nan)
            keep.append(True)
            continue

        box = [float(x1), float(y1), float(x2), float(y2)]

        if mode == "from_original":
            if per_image_src_size is None or img not in per_image_src_size:
                raise ValueError(f"Missing original size for image_id={img}")
            W0,H0 = per_image_src_size[img]
            # warp original -> 256
            sx1 = 256.0 / float(W0); sy1 = 256.0 / float(H0)
            box = warp_box(box, sx1, sy1)
            box = clamp_box(box, 256.0, 256.0)
            if box is None:
                if drop_degenerate: keep.append(False); continue
                else:
                    new_x1.append(np.nan); new_y1.append(np.nan)
                    new_x2.append(np.nan); new_y2.append(np.nan)
                    keep.append(True); continue

        elif mode != "from_256":
            raise ValueError("mode must be 'from_256' or 'from_original'")

        # warp 256 -> 224
        box = warp_box(box, scale_256_to_224, scale_256_to_224)
        box = clamp_box(box, w_target, h_target)
        if box is None:
            if drop_degenerate: keep.append(False); continue
            else:
                new_x1.append(np.nan); new_y1.append(np.nan)
                new_x2.append(np.nan); new_y2.append(np.nan)
                keep.append(True); continue

        x1,y1,x2,y2 = box
        new_x1.append(x1); new_y1.append(y1)
        new_x2.append(x2); new_y2.append(y2)
        keep.append(True)

    out = out.loc[keep].copy()
    out["x_min"] = new_x1[:len(out)]
    out["y_min"] = new_y1[:len(out)]
    out["x_max"] = new_x2[:len(out)]
    out["y_max"] = new_y2[:len(out)]
    return out

# --- Examples ---

# 1) Boxes already on 256 grid
# df_224 = transform_boxes_to_224(df, mode="from_256")

# 2) Boxes in original DICOM coords; you must supply per-image (W0,H0):
# per_image_src_size = dict(zip(meta_df.image_id, zip(meta_df.dim1, meta_df.dim0)))  # note: (W0,H0)
# df_224 = transform_boxes_to_224(df, mode="from_original", per_image_src_size=per_image_src_size)


In [5]:
%%time
meta_df = pd.read_csv('train.csv')
per_image_src_size = dict(zip(meta_df.image_id, zip(meta_df.width, meta_df.height)))  # note: (W0,H0)
df_224 = transform_boxes_to_224(meta_df, mode="from_original", per_image_src_size=per_image_src_size)

CPU times: user 344 ms, sys: 51.5 ms, total: 396 ms
Wall time: 401 ms


In [6]:
print(df_224)
df_224.to_csv("df_224.csv", index=False)

                               image_id          class_name  class_id rad_id  \
0      50a418190bc3fb1ef1633bf9678929b3          No finding        14    R11   
1      21a10246a5ec7af151081d0cd6d65dc9          No finding        14     R7   
2      9a5094b2563a1ef3ff50dc5c7ff71345        Cardiomegaly         3    R10   
3      051132a778e61a86eb147c7c6f564dfe  Aortic enlargement         0    R10   
4      063319de25ce7edb9b1c6b8881290140          No finding        14    R10   
...                                 ...                 ...       ...    ...   
67909  936fd5cff1c058d39817a08f58b72cae          No finding        14     R1   
67910  ca7e72954550eeb610fe22bf0244b7fa          No finding        14     R1   
67911  aa17d5312a0fb4a2939436abca7f9579          No finding        14     R8   
67912  4b56bc6d22b192f075f13231419dfcc8        Cardiomegaly         3     R8   
67913  5e272e3adbdaafb07a7e84a9e62b1a4c          No finding        14    R16   

            x_min       y_min       x_m

In [7]:
# split_vindr_like.py
import math
import os
import sys
import unicodedata
from collections import defaultdict, Counter

import numpy as np
import pandas as pd

# Try to use iterative stratification; fall back to a simple stratifier with a warning.
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    HAS_ITERSTRAT = True
except Exception:
    HAS_ITERSTRAT = False

NO_FINDING_ID = 14  # dataset convention

def nfc_norm(s: str) -> str:
    # NFC normalize + lowercase + strip spaces; "canonical" filename normalization
    return unicodedata.normalize("NFC", s).strip().lower()

def load_annotations(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # minimal schema checks
    need = {"image_id","class_id","rad_id","x_min","y_min","x_max","y_max"}
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"CSV missing columns: {miss}")
    # normalize IDs
    df["image_id"] = df["image_id"].astype(str).map(nfc_norm)
    df["rad_id"] = df["rad_id"].astype(str).str.strip()
    return df

def build_multilabel_targets(df: pd.DataFrame):
    """
    Build per-image targets.

    Majority rule per class c in [0..13]: positive if >=2 distinct raters annotated at least one bbox of class c.
    Soft label per class: (#distinct-raters-positive)/3 in {0, 1/3, 2/3, 1}.

    no_finding_majority: 1 if >=2 raters said No finding AND no other class has majority positive.
    """
    # Deduplicate boxes per (image, class, rater) so a rater with multiple boxes for same class counts once
    df_pos = df[df["class_id"] != NO_FINDING_ID].copy()
    df_pos = df_pos.dropna(subset=["x_min","y_min","x_max","y_max"])
    df_pos["rad_int"] = df_pos["rad_id"].str[1:].astype(int) if df_pos["rad_id"].str.startswith("R").all() else df_pos["rad_id"]

    # Rater -> class presence
    raters_per_class = (
        df_pos.groupby(["image_id","class_id"])["rad_id"]
        .nunique()
        .rename("rater_count")
        .reset_index()
    )

    # Initialize label tables
    images = sorted(df["image_id"].unique())
    classes = list(range(0, NO_FINDING_ID))  # 0..13
    idx = pd.Index(images, name="image_id")
    y_major = pd.DataFrame(0, index=idx, columns=classes, dtype=np.int8)
    y_soft   = pd.DataFrame(0.0, index=idx, columns=classes, dtype=float)

    # Fill per-class labels
    for _, row in raters_per_class.iterrows():
        img, cid, rc = row["image_id"], int(row["class_id"]), int(row["rater_count"])
        if cid in classes:
            y_soft.at[img, cid] = min(1.0, rc / 3.0)  # in {1/3, 2/3, 1} (or 0 if absent)
            y_major.at[img, cid] = 1 if rc >= 2 else max(y_major.at[img, cid], 0)

    # Ensure zeros where absent
    y_soft = y_soft.fillna(0.0)
    y_major = y_major.fillna(0).astype(np.int8)

    # No finding counts per image (how many raters said "No finding")
    nf_counts = (
        df[df["class_id"] == NO_FINDING_ID]
        .groupby("image_id")["rad_id"].nunique()
        .reindex(idx, fill_value=0)
    )
    # no_finding majority is 1 only if ≥2 raters say No finding and NO other class has majority positive
    no_find_major = ((nf_counts >= 2) & (y_major.sum(axis=1) == 0)).astype(np.int8)

    # Auxiliary: number of (unique) boxes per image (any class, any rater)
    n_boxes = (
        df_pos.groupby("image_id")[["x_min","y_min","x_max","y_max","rad_id","class_id"]]
        .apply(lambda g: len(g))
        .reindex(idx, fill_value=0)
        .rename("n_boxes")
    )

    targets = {
        "y_major": y_major,         # (N, 14) ints 0/1
        "y_soft": y_soft,           # (N, 14) floats in [0,1]
        "no_finding_major": no_find_major,  # (N,) ints 0/1
        "n_boxes": n_boxes,         # (N,) ints
    }
    return targets

def make_strat_matrix(targets, rare_bins=True):
    """
    Build a multilabel stratification matrix Z (N, K).

    Columns include:
      - 14 binary majority labels (classes 0,1,2,..,13)
      - one 'no_finding' column
      - optional buckets of n_boxes to keep annotation complexity roughly matched
    """
    Z = targets["y_major"].copy()
    Z["no_finding"] = targets["no_finding_major"]

    if rare_bins:
        # Bucketize n_boxes (annotation complexity) as a weak feature
        nb = targets["n_boxes"].clip(upper=60)
        # simple bins: 0-3, 4-9, 10-19, >=20
        bins = pd.cut(nb, bins=[-1,3,9,19,1000], labels=["b0_3","b4_9","b10_19","b20p"])
        for lab in bins.cat.categories:
            Z[f"bin_{lab}"] = (bins == lab).astype(int)

    return Z.astype(int)

def iterative_split(X_ids: np.ndarray, Z: np.ndarray, test_size: float, seed: int):
    """
    Multilabel iterative stratification Train/Test split on unique image_ids.
    """
    if HAS_ITERSTRAT:
        msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        (train_idx, test_idx) = next(msss.split(X_ids, Z))
        return X_ids[train_idx], X_ids[test_idx]
    else:
        # Fallback: warn + do a simple stratified shuffle on a scalar surrogate (label count).
        print("[WARN] iterative-stratification not available; using surrogate stratification on label counts. "
              "Install `iterative-stratification` for better matching.", file=sys.stderr)
        rng = np.random.default_rng(seed)
        order = rng.permutation(len(X_ids))
        cut = int(round(len(X_ids) * (1.0 - test_size)))
        tr = X_ids[order[:cut]]
        te = X_ids[order[cut:]]
        return tr, te

def make_splits(ids: pd.Index, Z: pd.DataFrame, seed: int = 42):
    """
    60/10/30 on unique image_ids:
      1) First take 30% Test
      2) From remaining 70%, take 10/70 ≈ 14.2857% as Val to achieve 10% overall
    """
    X = ids.to_numpy()
    Zm = Z.loc[ids].to_numpy(dtype=int)

    # Step 1: Test
    train_ids, test_ids = iterative_split(X, Zm, test_size=0.30, seed=seed)

    # Step 2: Val from remaining
    remain_mask = pd.Index(X).isin(test_ids) == False
    X_remain = X[remain_mask]
    Z_remain = Z.loc[X_remain].to_numpy(dtype=int)
    val_frac_of_remaining = 0.10 / 0.70  # == 1/7 ≈ 0.142857...
    train_ids2, val_ids = iterative_split(X_remain, Z_remain, test_size=val_frac_of_remaining, seed=seed+1)

    # Sanity
    assert len(set(train_ids2) & set(val_ids)) == 0
    assert len(set(test_ids) & set(val_ids)) == 0
    assert len(set(test_ids) & set(train_ids2)) == 0

    split = pd.Series(index=ids, data="train")
    split.loc[val_ids] = "val"
    split.loc[test_ids] = "test"
    return split

def audit_report(split_s: pd.Series, targets):
    y = targets["y_major"]
    nof = targets["no_finding_major"]
    nb = targets["n_boxes"]
    out = []

    for s in ["train","val","test"]:
        mask = (split_s == s)
        n = int(mask.sum())
        out.append(f"[{s}] N={n}")
        # per-class positives
        pos = y.loc[mask].sum()
        prev = (pos / n).round(4)
        lines = ", ".join([f"c{cid}:{int(pos[cid])} ({prev[cid]:.3f})" for cid in y.columns])
        out.append("  per-class positives: " + lines)
        nf_prev = float(nof.loc[mask].mean())
        out.append(f"  no_finding prev: {nf_prev:.3f}")
        # annotation complexity
        hist = nb.loc[mask].value_counts(bins=[-1,3,9,19,1000]).sort_index()
        out.append("  n_boxes bins: " + ", ".join([f"{str(i)}:{int(v)}" for i,v in hist.items()]))

    return "\n".join(out)

def main(csv_path: str, out_dir: str = "./splits_artifacts", seed: int = 42):
    os.makedirs(out_dir, exist_ok=True)
    df = load_annotations(csv_path)

    # Hard data integrity checks tied to dataset design
    # 1) exactly 3 raters per image overall (counting No finding rows too)?
    raters_per_image = df.groupby("image_id")["rad_id"].nunique()
    if not raters_per_image.between(3,3).all():
        bad = raters_per_image[raters_per_image != 3]
        raise AssertionError(f"Images without exactly 3 raters detected (n={len(bad)}). Example:\n{bad.head()}")

    # 2) 'No finding' rows must have NaN boxes; positives must have numeric boxes
    nf_bad = df[(df["class_id"] == NO_FINDING_ID) & (~df[["x_min","y_min","x_max","y_max"]].isna().all(axis=1))]
    pos_bad = df[(df["class_id"] != NO_FINDING_ID) & (df[["x_min","y_min","x_max","y_max"]].isna().any(axis=1))]
    if len(nf_bad):
        raise AssertionError(f"'No finding' rows with non-NaN boxes found: {len(nf_bad)}")
    if len(pos_bad):
        raise AssertionError(f"Positive rows with NaN bbox fields found: {len(pos_bad)}")

    # Build targets
    targets = build_multilabel_targets(df)
    ids = targets["y_major"].index
    Z = make_strat_matrix(targets, rare_bins=True)

    # Splits
    split_s = make_splits(ids, Z, seed=seed)

    # Artifacts
    patients = pd.DataFrame({
        "image_id": ids,
        "hospital": pd.Series(index=ids, dtype="object")  # fill if you have site metadata
    }).reset_index(drop=True)
    patients.to_csv(os.path.join(out_dir, "patients.csv"), index=False)

    splits = split_s.reset_index().rename(columns={0:"split"})
    splits.columns = ["image_id","split"]
    splits.to_csv(os.path.join(out_dir, "splits.csv"), index=False)

    # Audits
    rep = audit_report(split_s, targets)
    with open(os.path.join(out_dir, "join_report.md"), "w", encoding="utf-8") as f:
        f.write("# Split and Join Audit\n\n")
        f.write(rep + "\n")

    print(rep)
    print(f"\nWrote patients.csv and splits.csv to {out_dir}")


In [8]:
csv_path = "/workspace/df_224.csv"
out_dir = "./splits_artifacts"
seed = 42
main(csv_path, out_dir, seed)

[train] N=9000
  per-class positives: c0:1407 (0.156), c1:37 (0.004), c2:106 (0.012), c3:1090 (0.121), c4:73 (0.008), c5:91 (0.010), c6:147 (0.016), c7:328 (0.036), c8:243 (0.027), c9:217 (0.024), c10:381 (0.042), c11:529 (0.059), c12:35 (0.004), c13:610 (0.068)
  no_finding prev: 0.707
  n_boxes bins: (-1.001, 3.0]:6585, (3.0, 9.0]:1693, (9.0, 19.0]:631, (19.0, 1000.0]:91
[val] N=1500
  per-class positives: c0:235 (0.157), c1:6 (0.004), c2:18 (0.012), c3:182 (0.121), c4:12 (0.008), c5:15 (0.010), c6:25 (0.017), c7:55 (0.037), c8:41 (0.027), c9:36 (0.024), c10:63 (0.042), c11:88 (0.059), c12:6 (0.004), c13:102 (0.068)
  no_finding prev: 0.707
  n_boxes bins: (-1.001, 3.0]:1098, (3.0, 9.0]:282, (9.0, 19.0]:105, (19.0, 1000.0]:15
[test] N=4500
  per-class positives: c0:704 (0.156), c1:19 (0.004), c2:53 (0.012), c3:545 (0.121), c4:36 (0.008), c5:46 (0.010), c6:73 (0.016), c7:164 (0.036), c8:121 (0.027), c9:109 (0.024), c10:190 (0.042), c11:265 (0.059), c12:17 (0.004), c13:305 (0.068)
  no